In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import csv
import datetime

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) #ez hogy a rákban nem a defaultja a jupyternek?

# saját függvényeink
from functions import *

In [2]:
bipartite_data = createNxGraph()
projected_graph = listMEPs()
mep_social_network = nx.algorithms.bipartite.projected_graph(bipartite_data, projected_graph)

# SPLITTING THE DATA

In [27]:
import datetime

In [28]:
ENTIRE_DATASET["Date"] = pd.to_datetime(ENTIRE_DATASET["Date"], format = "%d/%m/%Y")

In [35]:
# https://stackoverflow.com/questions/55268578/split-dataframe-by-month

# groupby your key and freq
g = ENTIRE_DATASET.groupby(pd.Grouper(key='Date', freq='M'))
# groups to a list of dataframes with list comprehension
split_data = [group for _,group in g]

In [36]:
## clean this shit up
pd.options.mode.chained_assignment = None  # default='warn'
for i in split_data:
    i.drop(columns=["Committee", "Dossier", "DocumentType", "PENumber"], inplace = True)
pd.options.mode.chained_assignment = 'warn'


In [38]:
## we save the split data into ./db/split

for i, x in enumerate(split_data):
    x.to_csv(f"./db/split/{i}.csv")

^- Ennyi adatpontunk van intervallumonként. Nekem balanceoltnak tűnik őszintén szólva.

TODO: ezt a lentebbi analízist új notebookban kéne

Most megcsináljuk basically ugyanazt az analízist mint fönt, csak minden intervallumra külön.

In [8]:
def listMEPs_div(GDataset): # listing the different MEP names for the given dataset
    o = pd.unique(GDataset['MEPName'])
    return o
ListsMEP = [set(listMEPs_div(miniGraph)) for miniGraph in split_data]#list for each subgraph
#print(ListsMEP)
meps_who_did_something_in_all_intervals = set.intersection(*ListsMEP)
print(meps_who_did_something_in_all_intervals)
print(len(meps_who_did_something_in_all_intervals))

set()
0


In [9]:
by_country = {}
by_polgroup = {}
for mep in meps_who_did_something_in_all_intervals:
    curr_country = getMEPData("MEPName", mep)["MemberState"]
    curr_polgroup = getMEPData("MEPName", mep)["EPGroup"]
    if curr_country not in by_country:
        by_country[curr_country] = 0
    if curr_polgroup not in by_polgroup:
        by_polgroup[curr_polgroup] = 0
    by_country[curr_country] += 1
    by_polgroup[curr_polgroup] += 1

print(by_country)
print(by_polgroup)
    

{}
{}
